<a href="https://colab.research.google.com/github/akhilkapil/ANN-SLP-For-Multi-Classification/blob/master/ANN_SLP_For_Multiclassification_Using_Softmax_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Data/zoo.csv')
data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [ ]:
#Spliting the target and independent variables
X = data.iloc[:,:-1]
y = data.iloc[:,-1:]
print(y)

     class_type
0             1
1             1
2             4
3             1
4             1
..          ...
96            1
97            6
98            1
99            7
100           2

[101 rows x 1 columns]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

In [ ]:
#Drop the animals animal column from both the dataset
train_name = X_train['animal_name']
test_name = X_test['animal_name']

X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]

print("Training Data has",X_train.shape)
print("Testing Data has",X_test.shape)

Training Data has (70, 16)
Testing Data has (31, 16)


Make ANN/SLP *Model*

In [ ]:
#Lets make placeholders for dynamic variable allocation
import tensorflow as tf

In [ ]:
tf.compat.v1.disable_eager_execution()

#Lets make placeholders for dynamic variable allocation

Placeholder is one of the function in tensorflow. It is a space to put and change values while the program is running.

for X, a place must have 16 columns, since zoo data has 16 features.

for Y, a place must have 1 columns, since the results has 1 outcome.

If you see the row "None", it means it has no size limits. (You can write -1 instead of "None")

In [ ]:
import tensorflow as tf

X = tf.compat.v1.placeholder(tf.float32,[None, 16])
Y = tf.compat.v1.placeholder(tf.int32,[None, 1])
print(X)

Tensor("Placeholder_12:0", shape=(None, 16), dtype=float32)


In [ ]:
#We have to change our target varibale to one-hot-encoding as it will be much easier for our machine to learn from it 
y_encoded = tf.compat.v1.one_hot(Y, 7)
y_encoded = tf.reshape(y_encoded, shape=[-1,7])
print(y_encoded)

Tensor("Reshape_5:0", shape=(None, 7), dtype=float32)


In [ ]:
#Make Weigh, Bias variable with random values only

W = tf.compat.v1.Variable(tf.compat.v1.random_normal([16,7], seed=0), name='weight')
B = tf.compat.v1.Variable(tf.compat.v1.random_normal([7], seed=0), name='bias')


In [ ]:
#Make output result
logits = tf.compat.v1.matmul(X,W) + B
hypothesis = tf.nn.softmax(logits)
cost_i = tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_encoded)
cost = tf.reduce_mean(cost_i)
print(cost_i)

Tensor("softmax_cross_entropy_with_logits_6/Reshape_2:0", shape=(None,), dtype=float32)


In [ ]:
#Now our cost function is ready now we will apply our optimizer

train =tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss=cost)

In [ ]:
 #Now Lets compare Original Vs Prediction

 prediction = tf.argmax(hypothesis,1)
 correct_predictions = tf.equal(prediction, tf.argmax(y_encoded,1))
 accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [ ]:
#Activate the model

with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for step in range(5001):
    sess.run(train, feed_dict={X:X_train, Y:y_train})
    if step % 1000 == 0:
      loss, acc = sess.run([cost,accuracy], feed_dict={X: X_train, Y: y_train})
      print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
      train_acc = sess.run(accuracy, feed_dict={X:X_train, Y: y_train})
      test_acc, test_predict, test_correct = sess.run([accuracy,prediction,correct_predictions], feed_dict={X:X_train, Y: y_train})
      print('Train predictions: ', train_acc)
      print('Test predictions: ', test_acc)

Step:     0	Loss: 3.391	Acc: 27.14%
Train predictions:  0.27142859
Test predictions:  0.27142859
Step:  1000	Loss: 0.110	Acc: 90.00%
Train predictions:  0.9
Test predictions:  0.9
Step:  2000	Loss: 0.054	Acc: 91.43%
Train predictions:  0.9142857
Test predictions:  0.9142857
Step:  3000	Loss: 0.036	Acc: 91.43%
Train predictions:  0.9142857
Test predictions:  0.9142857
Step:  4000	Loss: 0.028	Acc: 91.43%
Train predictions:  0.9142857
Test predictions:  0.9142857
Step:  5000	Loss: 0.023	Acc: 91.43%
Train predictions:  0.9142857
Test predictions:  0.9142857


In [ ]:
test_predict

array([1, 1, 4, 4, 1, 2, 6, 1, 1, 1, 2, 4, 4, 1, 2, 1, 4, 2, 4, 4, 1, 3,
       1, 6, 1, 6, 5, 1, 1, 3, 1, 2, 6, 2, 2, 5, 3, 1, 2, 4, 1, 1, 6, 1,
       1, 5, 3, 6, 4, 6, 4, 2, 6, 2, 0, 1, 2, 1, 1, 2, 5, 2, 1, 2, 1, 6,
       1, 1, 1, 1])